# Configurations

In [ ]:
# These are configuration variable used by the notebook

# Category Map from Nubank/Splitwise to official categories
# This map will be used to convert a category to another one
CATEGORY_CONVERSION_TABLE = {
    'Eletricidade': 'Casa',
    'Aluguel': 'Casa',
    'Móveis': 'Casa',
    'Jantar fora': 'Restaurante',
    'Supermercado': 'Mercado',
    'supermercado': 'mercado',
    'Eletrônicos': 'Compras',
    'Vestuário': 'Compras',
    'Produtos de limpeza': 'Compras',
    'Filmes': 'Lazer',
    'Música': 'Lazer',
    'Bicicleta': 'Lazer',
    'TV/Telefone/Internet': 'Serviços',
    'Despesas médicas': 'Saúde',
    'Táxi': 'Transporte',
    'Ônibus/trem': 'Viagem',
    'Avião': 'Viagem',
    'Carro': 'Viagem',
    'Combustível': 'Viagem',
    'Hotel': 'Viagem',
    'Presentes': 'Outros',
    'Geral': 'Outros',
    'Ajuste': 'Outros',
    'Manutenção': 'Compras',
    'Jogos': 'Lazer',
    'Seguro': 'Outros'
}


LOAD_CONFIGS = {
    'data_path': "./data", # Directory that contains the expenses data
    'nubank_file_pattern': "nubank*.csv",
    'splitwise_file_pattern': "20*mozi-e-eu*.csv",
    'manual_file_pattern': "Extrato outras contas*Despesas*.csv",
    'incomes_file_pattern': "Extrato outras contas*Rendimentos*.csv",
    'person_who_pays': "Lucas Alencar", # Person name that the expenses will be extracted from Splitwise csv
    'category_conversion_table': CATEGORY_CONVERSION_TABLE,
}


from datetime import date
from date_helpers import previous_month

# Date used to make month analysis through the notebook
# BASE_DATE = date(2018, 7, 1)
BASE_DATE = previous_month(date.today())

# Year you were born
YEAR_OF_BIRTH = 1991

# Data load and preprocess

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

%matplotlib inline

from IPython.display import display
from record_summary import *
from date_helpers import *

from load import incomes, expenses

incomes = incomes.load(**LOAD_CONFIGS)
expenses = expenses.load(**LOAD_CONFIGS)

# Monthly Expenses

In [ ]:
from monthly_expenses import *

MONTHLY_BALANCE_GOAL = 0.4

# Expenses and incomes by month
exp_by_month = total_amount_by(groupby_month(expenses), expenses)

income_salary = incomes[incomes.category == 'renda'][['date', 'amount']]
inc_by_month = total_amount_by(groupby_month(income_salary), income_salary)

monthly_exp = summary_expenses(exp_by_month, inc_by_month)

# Monthly averages
avg_monthly_exp = summary_expenses(exp_by_month.mean(), inc_by_month.mean())
avg_monthly_exp.index = ['Average monthly expenses']

print(" >>> Monthly balance goal:", MONTHLY_BALANCE_GOAL * 100, "%")
display(style_summary_expenses(monthly_exp, MONTHLY_BALANCE_GOAL))
display(style_summary_expenses(avg_monthly_exp, MONTHLY_BALANCE_GOAL))

plot_expenses_summary(monthly_exp)

# Expenses by category

In [ ]:
from monthly_expenses import *

# BASE_DATE = date(2018, 11, 1)
print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)

expenses_for_month = records_for_month(expenses, BASE_DATE)
incomes_for_month = records_for_month(incomes, BASE_DATE)

expenses_for_category_by_total = describe_expenses(expenses_for_month, incomes_for_month)
display(style_expenses_distribution(expenses_for_category_by_total))

def display_expenses_by_category(category):
    print("Expenses in category", category)
    display(expenses_for_month[expenses_for_month.category == category].sort_values('amount'))
    
display_expenses_by_category("outros")

In [ ]:
# Plots over time for each category grouping them by easy of read
expenses_over_time_data = expenses_over_time(expenses, incomes, 'amount #')

def food_expenses(expenses, base_date):
    data = total_amount_by('category', records_for_month(expenses, base_date))
    if 'mercado' in data.index and 'restaurante' in data.index:
        return pd.Series(data.loc['mercado'].amount + data.loc['restaurante'].amount) * -1
    else:
        return pd.Series(0)

expenses_over_time_data['alimentação'] = describe_over_time(expenses, food_expenses)

EXPENSES_FIGSIZE = (20, 5)

plot_over_time(expenses_over_time_data[['casa', 'viagem', 'educação']], figsize=EXPENSES_FIGSIZE)
plot_over_time(expenses_over_time_data[['restaurante', 'mercado', 'alimentação']], figsize=EXPENSES_FIGSIZE)
plot_over_time(expenses_over_time_data[['transporte', 'lazer', 'serviços']], figsize=EXPENSES_FIGSIZE)

# Return progress

In [ ]:
from monthly_investiments import *
from central_bank_data import central_bank_metric, BC_IPCA_BY_MONTH_ID


# BASE_DATE = date(2019, 1, 1)
print(">>> Month of analysis:", BASE_DATE.month, BASE_DATE.year)

current_ipca = central_bank_metric(BC_IPCA_BY_MONTH_ID, BASE_DATE)
current_ipca = current_ipca * 100 if current_ipca else None
print("> Current IPCA:", current_ipca, "%")

RETURN_FOR_MONTH_GOAL = 0.005
RETURN_WITH_INFLATION_GOAL = 0.0045

invest = incomes[incomes.category.isin(['aplicação', 'valor aplicado', 'desconto'])]

# Table with returns for the current month
summary_invest = summary_investments_current_month(invest, BASE_DATE)\
                 .sort_values('Return for month (%)', ascending=False)

display(style_summary_investments(summary_invest, RETURN_FOR_MONTH_GOAL, RETURN_WITH_INFLATION_GOAL))

# Return over time

In [ ]:
# Get all investments titles
# display(pd.DataFrame(invest['title'].unique(), columns=['title']))

# Some visualizations to use and see the return history for the groups
general = ['Nuconta Renda Fixa 100% CDI',
           'Tesouro Selic 2021',
           'CDB Banco Maxima',
           'Magnetis Diversificação Renda Fixa',
           'Magnetis Diversificação Multimercados',
           'Magnetis Diversificação Ações']

# Other investments in renda fixa are better than Tesouro Selic? They are still worth it?
renda_fixa = ['Nuconta Renda Fixa 100% CDI',
              'Tesouro Selic 2021',
              'Tesouro Selic 2023',
              'Magnetis Diversificação Renda Fixa',
              'LCI Banco Original',
              'CDB Banco Maxima',
              'Brasil Plural Yield Fundo de Investimento Renda Fixa Referen']

# Is Magnetis investing in good stocks? Is the cumulative return increasing?
renda_variavel = ['Magnetis Diversificação Ações',
                  'BRAX11',
                  'Magnetis Diversificação Multimercados']

# Does bitcoin is still a good experiment?
bitcoins = ['Bitcoin', 'Cripto Trade']

# There are better opportunities of investments over these Tesouros?
tesouros = ['Tesouro IPCA+ 2019',
            'Tesouro IPCA+ 2024']


RETURN_STARTING_MONTH = '2018-01'
RETURN_FIGSIZE = (20, 7)
PLOT_TITLE_GROUP = renda_fixa

plot_over_time(return_percentage_over_time(invest).loc[RETURN_STARTING_MONTH:, PLOT_TITLE_GROUP], 
               title='Return over time (%)', figsize=RETURN_FIGSIZE)

plot_over_time(cumulative_return_over_time(invest).loc[RETURN_STARTING_MONTH:, PLOT_TITLE_GROUP], 
               title='Cumulative return over time', figsize=RETURN_FIGSIZE)

# Investment progress

In [ ]:
assets_summary = summary_assets(invest)
display(style_summary_assets(assets_summary))

plot_invest_type_distribution(invest, BASE_DATE)

# Plot line to see assets progress
ASSETS_SUMMARY_START_DATE = '2018-02'

plot_over_time(assets_summary.loc[ASSETS_SUMMARY_START_DATE:, 'Return'], figsize=(20, 5))
plot_over_time(assets_summary.loc[ASSETS_SUMMARY_START_DATE:, 'Applications'], figsize=(20, 5))

# Investments liquidations

In [ ]:
finished = finished_invests(invest).reset_index().title
finished_investments = incomes[incomes.title.isin(list(finished))]
invests_final_return(finished_investments)

# Assets goals

In [ ]:
# PMS Patrimônio Mínimo de Sobrevivência (Reserva de emergência)
# 6 * Custo mensal

# PMR Patrimônio Mínimo Recomendado para segurança 
# Possibilita decisões mais arriscada sobre carreira ou mudanças.
# 12 * Custo mensal

# PI Patrimônio Ideal para idade e consumo 
# Dado os meus gastos mensais atuais, se eu continuar trabalhando até a aposentadoria (65 anos), 
# eu conseguiria viver apenas gastando esse dinheiro.
# 0.1 * PMR * Idade

# PNIF Patrimônio Necessário para a Independência Financeira
# Dado os meus rendimentos com investimentos, eu conseguiria manter o mesmo padrão apenas com o dinheiro investido
# PMR / Rendimento anual %

from datetime import date
from monthly_investiments import *
from investments import totals as tt

age = date.today().year - YEAR_OF_BIRTH

# This is the annualized return based on the return for every month
# It is starting to count the return beginning on 2018-04, 
# because this is the most accurate values I have now
annualized_return = assets_summary.loc['2018-04':, 'Return / Total'].mean() * 12
print("Annualized return:", annualized_return * 100, '%')

pms = 6 * exp_by_month.mean() * -1
pmr = 12 * exp_by_month.mean() * -1
pi = 0.1 * pmr * age
pnif = pmr / annualized_return
assets_total = tt.invested_for_month_by('title', invest, BASE_DATE).sum()

assets_goals = pd.DataFrame([pms, pmr, pi, pnif, assets_total], index=['PMS', 'PMR', 'PI', 'PNIF', 'Total'])
display(style_assets_goals(assets_goals, assets_total.amount))

plot_over_time(assets_summary.loc['2018-01':, 'Total'], figsize=(20, 5))

# How much my time is worth?

In [ ]:
def weekdays_in_month(BASE_DATE):
    month_calendar = calendar.monthcalendar(BASE_DATE.year, BASE_DATE.month)
    weekdays_in_month = 0
    for week in month_calendar:
        for day in filter(lambda x: x > 0, week):
            if calendar.weekday(BASE_DATE.year, BASE_DATE.month, day) < 5:
                weekdays_in_month += 1
    return weekdays_in_month

# BASE_DATE = date(2018, 6, 1)

total_earned_on_base_date = records_for_month(incomes[incomes.category == 'renda'], BASE_DATE).amount.sum()
total_worked_by_day = 8.8 # According to CLT
total_worked_by_month = total_worked_by_day * weekdays_in_month(BASE_DATE)

money_for_time = total_earned_on_base_date / total_worked_by_month

print("In", BASE_DATE.strftime('%Y-%m'), "you have earned", total_earned_on_base_date, ".")
print("Given you have worked for {:,.2f} hours in this month.".format(total_worked_by_month))
print("Your time is worth {:,.2f} R$/hour.".format(money_for_time), "Not considering holidays or vacations.")

# TODO

- Add index to measure my financial independence
- Add analysis when an investment has ended. How much did I win with it and if it was worth it.